In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sympy as sym

plt.rcParams['animation.html'] = 'jshtml'

In [ ]:
def heaviside(x):
    return np.heaviside(x, .5)

def animate_convolution(x, h, y, td, taud):

    # add numerical evaluation to symbolic functions
    x_eval = sym.lambdify((tau), x.subs(t, tau), modules=['numpy', {'Heaviside': heaviside}])
    h_eval = sym.lambdify((t, tau), h.subs(t, t - tau), modules=['numpy', {'Heaviside': heaviside}])
    y_eval = sym.lambdify((t), y, modules=['numpy', {'Heaviside': heaviside}])

    # setup plot and line styles
    fig, ax = plt.subplots(2, 1)
    fig.subplots_adjust(hspace=0.5)
    plt.close()  # suppresses empty plot in notebook

    lines = [ax[0].plot([], [], lw=2, label=r'$h(t-\tau)$')[0]]
    lines.append(ax[0].plot([], [], lw=2, label=r'$x(\tau)$')[0])
    lines.append(ax[1].plot([], [], 'g-', lw=2, label=r'$y(t) = x(t) * h(t)$')[0])
    lines.append(ax[1].plot([], [], 'ro', lw=2)[0])

    ax[0].set_xlim((-3, 5))
    ax[0].set_ylim((-.1, 1.2))
    ax[0].set_xlabel(r'$\tau$')
    ax[0].legend(loc='upper right')
    ax[0].grid(True)

    ax[1].set_xlim((-3, 5))
    ax[1].set_ylim((-.1, 1.2))
    ax[1].set_xlabel(r'$t$')
    ax[1].legend(loc='upper right')
    ax[1].grid(True)

    
    def animate(ti):
        lines[0].set_data(taud, h_eval(ti, taud))
        lines[1].set_data(taud, x_eval(taud))

        lines[2].set_data(taud, y_eval(taud))
        lines[3].set_data(ti, y_eval(ti))
        
        ax[0].collections.clear()
        ax[0].fill_between(taud, 0, h_eval(ti, taud)*x_eval(taud), facecolor='green', alpha=0.5)

        return lines

    return animation.FuncAnimation(fig, animate, td, interval=50, blit=True)

# Examples

In [ ]:
class rect(sym.Function):

    @classmethod
    def eval(cls, arg):
        return sym.Heaviside(arg + sym.S.Half) - sym.Heaviside(arg - sym.S.Half)

t, tau = sym.symbols('t tau', real=True)

In [ ]:
h = sym.exp(-t) * sym.Heaviside(t)
x = rect(t - 1/2)
y = sym.integrate(h.subs(t,t-tau) * x.subs(t, tau), (tau, 0, t))  # for causal signals

anim = animate_convolution(x, h, y, np.arange(-2, 5, .1), np.arange(-10, 10, 0.01))
anim

In [ ]:
h = rect(t/2 - 1/2)
x = rect(t - 1/2)
y = sym.integrate(h.subs(t,t-tau) * x.subs(t, tau), (tau, 0, t))  # for causal signals

anim = animate_convolution(x, h, y, np.arange(-2, 5, .1), np.arange(-10, 10, 0.01))
anim